# 03 Bayesian Ranking - Core System

Build and test the Bayesian ranking system on a single race.

## What's In This Notebook

1. **Prior generation** - Convert championship standings to Bayesian priors
2. **Bayesian ranking system** - Update priors with evidence
3. **Performance scoring** - Extract driver rankings from telemetry
4. **Single race test** - Validate on Bahrain 2024
5. **Temporal validation** - Prove we're not cheating with future data

**Expected result:** MAE ~2.6-3.0 on single race


In [1]:
import pandas as pd
import numpy as np
import fastf1
from pathlib import Path
from dataclasses import dataclass
from typing import Dict
import warnings
warnings.filterwarnings('ignore')

# Setup
cache_dir = Path('../data/raw/.fastf1_cache')
fastf1.Cache.enable_cache(str(cache_dir))

print("="*80)
print("BAYESIAN VALIDATION: 2025 Season")
print("="*80)

BAYESIAN VALIDATION: 2025 Season


## PART 1: Tier-Based Priors (Realistic for 2026)

In [2]:
print("\n" + "="*80)
print("PART 1: Define Team/Driver Tiers")
print("="*80)

# For 2026, we'd know organizational capability, not exact performance
# Let's simulate this with 2025 data

@dataclass
class DriverPrior:
    """Prior belief about driver performance."""
    driver_number: str
    driver_code: str
    team: str
    team_tier: str  # 'top', 'midfield', 'backmarker'
    driver_tier: str  # 'elite', 'experienced', 'rookie'
    
    # Initial rating (μ, σ)
    # μ = expected position (higher = better, 20 = best, 1 = worst)
    # σ = uncertainty (higher = more uncertain)
    mu: float
    sigma: float

def initialize_2026_style_priors() -> Dict[str, DriverPrior]:
    """
    Initialize priors based on 2025 championship standings.
    
    2025 Results inform tier structure:
    - Top 4: NOR(423), VER(421), PIA(410), RUS(319) - Clear elite
    - Next tier: LEC(242), HAM(156), ANT(150) - Strong but gap to top
    - Midfield: ALB(73), SAI(64), ALO(56), HUL(51), HAD(51)
    - Lower: BEA(41), LAW(38), OCO(38), STR(33), TSU(33)
    - Backmarkers: GAS(22), BOR(19), COL(0)
    
    For 2026: Keep tier structure but HIGH uncertainty (new regs)
    """
    priors = {
        # TOP TIER - 2025 Top 4 (clear elite)
        '4': DriverPrior('4', 'NOR', 'McLaren', 'top', 'elite', mu=18, sigma=4),  # Champion
        '1': DriverPrior('1', 'VER', 'Red Bull Racing', 'top', 'elite', mu=18, sigma=4),  # 2 pts behind!
        '81': DriverPrior('81', 'PIA', 'McLaren', 'top', 'elite', mu=17, sigma=4),  # Strong 3rd
        '63': DriverPrior('63', 'RUS', 'Mercedes', 'top', 'elite', mu=17, sigma=4),  # Clear 4th
        
        # UPPER MIDFIELD - Next tier (strong but gap to top)
        '16': DriverPrior('16', 'LEC', 'Ferrari', 'midfield', 'elite', mu=14, sigma=5),  # 242 pts
        '44': DriverPrior('44', 'HAM', 'Ferrari', 'midfield', 'elite', mu=13, sigma=5),  # 156 pts (struggled)
        '12': DriverPrior('12', 'ANT', 'Mercedes', 'midfield', 'rookie', mu=13, sigma=5),  # 150 pts (strong rookie)
        
        # MIDFIELD - Best of rest
        '23': DriverPrior('23', 'ALB', 'Williams', 'midfield', 'experienced', mu=11, sigma=5),  # 73 pts
        '55': DriverPrior('55', 'SAI', 'Williams', 'midfield', 'experienced', mu=11, sigma=5),  # 64 pts
        '14': DriverPrior('14', 'ALO', 'Aston Martin', 'midfield', 'elite', mu=11, sigma=5),  # 56 pts
        '27': DriverPrior('27', 'HUL', 'Kick Sauber', 'midfield', 'experienced', mu=10, sigma=6),  # 51 pts
        '6': DriverPrior('6', 'HAD', 'Racing Bulls', 'midfield', 'rookie', mu=10, sigma=6),  # 51 pts (good rookie)
        
        # LOWER MIDFIELD
        '87': DriverPrior('87', 'BEA', 'Haas F1 Team', 'backmarker', 'rookie', mu=9, sigma=6),  # 41 pts
        '30': DriverPrior('30', 'LAW', 'Racing Bulls', 'backmarker', 'experienced', mu=9, sigma=6),  # 38 pts
        '31': DriverPrior('31', 'OCO', 'Haas F1 Team', 'backmarker', 'experienced', mu=9, sigma=6),  # 38 pts
        '18': DriverPrior('18', 'STR', 'Aston Martin', 'backmarker', 'experienced', mu=8, sigma=6),  # 33 pts
        '22': DriverPrior('22', 'TSU', 'Red Bull Racing', 'backmarker', 'experienced', mu=8, sigma=6),  # 33 pts (struggled)
        
        # BACKMARKERS
        '10': DriverPrior('10', 'GAS', 'Alpine', 'backmarker', 'experienced', mu=7, sigma=6),  # 22 pts
        '5': DriverPrior('5', 'BOR', 'Kick Sauber', 'backmarker', 'rookie', mu=6, sigma=7),  # 19 pts
        '7': DriverPrior('7', 'COL', 'Alpine', 'backmarker', 'rookie', mu=5, sigma=7),  # 0 pts (replaced DOO)
    }
    
    return priors

# Initialize priors
priors = initialize_2026_style_priors()

print("\nInitialized priors for 20 drivers:")
print(f"\nTeam Tiers:")
tier_counts = {}
for p in priors.values():
    tier_counts[p.team_tier] = tier_counts.get(p.team_tier, 0) + 1

for tier, count in sorted(tier_counts.items(), key=lambda x: x[1], reverse=True):
    print(f"  {tier:12} {count} drivers")

print(f"\nSample priors:")
for driver_num in ['1', '4', '16', '27']:
    p = priors[driver_num]
    print(f"  {p.driver_code:3} ({p.team:12}): μ={p.mu:4.1f}, σ={p.sigma:3.1f} [{p.team_tier}]")

print("\n🟢 Priors defined")
print("\nKey properties:")
print("  - High uncertainty (σ=4-7) reflects 2026 regulation change")
print("  - Top teams: μ=15-18 (expected P3-P5 range)")
print("  - Backmarkers: μ=3-6 (expected P15-P18 range)")
print("  - NO exact positions - just tier expectations")


PART 1: Define Team/Driver Tiers

Initialized priors for 20 drivers:

Team Tiers:
  midfield     8 drivers
  backmarker   8 drivers
  top          4 drivers

Sample priors:
  VER (Red Bull Racing): μ=18.0, σ=4.0 [top]
  NOR (McLaren     ): μ=18.0, σ=4.0 [top]
  LEC (Ferrari     ): μ=14.0, σ=5.0 [midfield]
  HUL (Kick Sauber ): μ=10.0, σ=6.0 [midfield]

🟢 Priors defined

Key properties:
  - High uncertainty (σ=4-7) reflects 2026 regulation change
  - Top teams: μ=15-18 (expected P3-P5 range)
  - Backmarkers: μ=3-6 (expected P15-P18 range)
  - NO exact positions - just tier expectations


## PART 2: Bayesian Update Mechanism


In [3]:
print("\n" + "="*80)
print("PART 2: Bayesian Ranking System")
print("="*80)

class BayesianDriverRanking:
    """
    Bayesian ranking system with Gaussian priors.
    
    Each driver has rating ~ N(μ, σ²)
    - μ = performance rating (higher = better)
    - σ = uncertainty (decreases as I observe more)
    
    Update rule: Bayesian inference with observed positions
    """
    
    def __init__(self, priors: Dict[str, DriverPrior]):
        """
        Initialize with prior beliefs.
        
        Parameters
        ----------
        priors : dict
            driver_number -> DriverPrior
        """
        self.ratings = {}  # driver_number -> (mu, sigma)
        self.n_observations = {}  # driver_number -> count
        
        for driver_num, prior in priors.items():
            self.ratings[driver_num] = (prior.mu, prior.sigma)
            self.n_observations[driver_num] = 0
        
        self.priors = priors
        self.update_history = []  # Track all updates
    
    def predict_positions(self) -> pd.DataFrame:
        """
        Predict positions based on current ratings.
        
        Returns
        -------
        pd.DataFrame
            Columns: driver_number, predicted_position, ci_lower, ci_upper
            Sorted by predicted position (best first)
        """
        predictions = []
        
        for driver_num, (mu, sigma) in self.ratings.items():
            # Convert rating to position (invert scale)
            # Rating 18 → Position ~2-3
            # Rating 10 → Position ~10-11
            # Rating 3  → Position ~18-19
            
            # Simple linear mapping: position = 21 - rating
            predicted_pos = 21 - mu
            
            # Confidence interval (95% = ±1.96σ)
            ci_lower = max(1, predicted_pos - 1.96 * sigma)
            ci_upper = min(20, predicted_pos + 1.96 * sigma)
            
            predictions.append({
                'driver_number': driver_num,
                'driver_code': self.priors[driver_num].driver_code,
                'team': self.priors[driver_num].team,
                'rating_mu': mu,
                'rating_sigma': sigma,
                'predicted_position': predicted_pos,
                'ci_lower': ci_lower,
                'ci_upper': ci_upper,
                'n_observations': self.n_observations[driver_num]
            })
        
        df = pd.DataFrame(predictions)
        df = df.sort_values('predicted_position')
        df['predicted_rank'] = range(1, len(df) + 1)
        
        return df
    
    def update_from_session(
        self,
        observed_positions: Dict[str, int],
        confidence_weight: float = 1.0,
        session_name: str = "Session"
    ):
        """
        Update ratings based on observed positions.
        
        Uses Bayesian update with Gaussian likelihood.
        
        Parameters
        ----------
        observed_positions : dict
            driver_number -> finishing position (1-20)
        confidence_weight : float
            How much to trust this observation
            - 0.1 = testing (low trust)
            - 0.3 = practice (medium trust)
            - 0.7 = qualifying (high trust)
            - 1.0 = race (full trust)
        session_name : str
            For logging
        """
        # Bayesian update: posterior = likelihood × prior
        # For Gaussian: new_sigma² = 1 / (1/prior_sigma² + 1/obs_sigma²)
        #               new_mu = (prior_mu/prior_sigma² + obs_mu/obs_sigma²) × new_sigma²
        
        for driver_num, observed_pos in observed_positions.items():
            if driver_num not in self.ratings:
                continue  # Skip unknown drivers
            
            prior_mu, prior_sigma = self.ratings[driver_num]
            
            # Convert observed position to rating
            # Position 1 → Rating ~19-20
            # Position 10 → Rating ~10-11
            # Position 20 → Rating ~1-2
            observed_rating = 21 - observed_pos
            
            # Observation uncertainty (inversely proportional to confidence)
            # Higher confidence = lower observation uncertainty
            obs_sigma = 5.0 / confidence_weight  # Range: 5 (race) to 50 (testing)
            
            # Bayesian update
            new_sigma_sq = 1 / (1/prior_sigma**2 + 1/obs_sigma**2)
            new_sigma = np.sqrt(new_sigma_sq)
            
            new_mu = (prior_mu/prior_sigma**2 + observed_rating/obs_sigma**2) * new_sigma_sq
            
            # Update ratings
            self.ratings[driver_num] = (new_mu, new_sigma)
            self.n_observations[driver_num] += 1
            
            # Log update
            self.update_history.append({
                'session': session_name,
                'driver_number': driver_num,
                'driver_code': self.priors[driver_num].driver_code,
                'observed_position': observed_pos,
                'prior_mu': prior_mu,
                'prior_sigma': prior_sigma,
                'new_mu': new_mu,
                'new_sigma': new_sigma,
                'confidence_weight': confidence_weight
            })
    
    def get_update_summary(self) -> pd.DataFrame:
        """Get summary of all updates."""
        return pd.DataFrame(self.update_history)


# TEST: Initialize and Make Initial Prediction

print("\nInitializing Bayesian ranking system...")
model = BayesianDriverRanking(priors)

print("🟢 Model initialized with 20 drivers")

# Make initial prediction (before any data)
initial_predictions = model.predict_positions()

print("\n" + "="*80)
print("INITIAL PREDICTIONS (Before Any Data)")
print("="*80)

print("\nTop 10 predicted:")
print(initial_predictions.head(10)[
    ['predicted_rank', 'driver_code', 'team', 'predicted_position', 'ci_lower', 'ci_upper', 'rating_sigma']
].to_string(index=False))

print("\nBottom 5 predicted:")
print(initial_predictions.tail(5)[
    ['predicted_rank', 'driver_code', 'team', 'predicted_position', 'ci_lower', 'ci_upper', 'rating_sigma']
].to_string(index=False))

print("\n" + "="*80)
print("Key Observations:")
print("="*80)
print(f"  - Predictions based purely on team/driver tiers")
print(f"  - Average CI width: {(initial_predictions['ci_upper'] - initial_predictions['ci_lower']).mean():.1f} positions")
print(f"  - Max uncertainty: {initial_predictions['rating_sigma'].max():.1f}")
print(f"  - This reflects: 'I know tiers but not exact order'")

print("\n🟢 Bayesian update mechanism ready")


PART 2: Bayesian Ranking System

Initializing Bayesian ranking system...
🟢 Model initialized with 20 drivers

INITIAL PREDICTIONS (Before Any Data)

Top 10 predicted:
 predicted_rank driver_code            team  predicted_position  ci_lower  ci_upper  rating_sigma
              1         NOR         McLaren                   3       1.0     10.84             4
              2         VER Red Bull Racing                   3       1.0     10.84             4
              3         PIA         McLaren                   4       1.0     11.84             4
              4         RUS        Mercedes                   4       1.0     11.84             4
              5         LEC         Ferrari                   7       1.0     16.80             5
              6         HAM         Ferrari                   8       1.0     17.80             5
              7         ANT        Mercedes                   8       1.0     17.80             5
              8         SAI        Williams     

## PART 3: Car Performance Profiles from Testing


In [4]:
# PART 3: Car Performance Profiles from Testing

class CarPerformanceProfile:
    """
    Extract car characteristics from telemetry features.
    
    Profile includes:
    - Slow corner performance (0-100 km/h)
    - Medium corner performance (100-200 km/h)
    - High corner performance (200+ km/h)
    - Straight-line speed (full throttle)
    - Tire degradation (lap time delta over stint)
    - Stability (throttle/brake smoothness)
    """
    
    def extract_from_testing(self, testing_features: pd.DataFrame) -> dict:
        """
        Convert testing features into car profile.
        
        Returns
        -------
        dict
            team -> {
                'slow_corner_advantage': float (km/h vs median),
                'medium_corner_advantage': float,
                'high_corner_advantage': float,
                'straight_advantage': float,
                'deg_index': float (lower = better),
                'stability_index': float (higher = better)
            }
        """
        pass  # We'll implement this

class TrackCharacteristics:
    """
    Database of track characteristics.
    
    For each circuit:
    - % of lap in slow corners
    - % of lap in medium corners  
    - % of lap in high corners
    - % of lap on straights
    - Avg lap time (for normalization)
    - Tire stress level
    """
    
    def load_track_database(self) -> pd.DataFrame:
        """Load track characteristics for all circuits."""
        pass  # We'll build this

class TrackSpecificPredictor:
    """
    Predict positions by matching car profiles to track demands.
    
    Logic:
    If McLaren has +8 km/h in medium corners
    And Silverstone is 60% medium corners
    Then McLaren gets performance boost at Silverstone
    """
    
    def predict_for_track(
        self,
        car_profiles: dict,
        track_chars: dict,
        bayesian_priors: dict
    ) -> pd.DataFrame:
        """
        Predict race results for specific track.
        
        Combines:
        1. Bayesian priors (team tier)
        2. Car performance profile (strengths/weaknesses)
        3. Track characteristics (demands)
        
        Returns predictions with uncertainty
        """
        pass  # We'll implement this

In [5]:
# Load 2024 testing features (from notebook 02)
testing_2024 = pd.read_parquet('../data/processed/testing_files/2024_testing_features.parquet')

print(f"🟢 Loaded {len(testing_2024)} driver-sessions")
print(f"  Columns: {len(testing_2024.columns)}")
print(f"\nAvailable corner speed columns:")
corner_cols = [c for c in testing_2024.columns if 'corner' in c.lower()]
print(f"  {corner_cols}")

🟢 Loaded 60 driver-sessions
  Columns: 76

Available corner speed columns:
  ['slow_corner_speed', 'medium_corner_speed', 'high_corner_speed', 'slow_corner_speed_std', 'medium_corner_speed_std', 'high_corner_speed_std', 'slow_corner_speed_rel', 'medium_corner_speed_rel', 'high_corner_speed_rel', 'slow_corner_speed_std_rel', 'medium_corner_speed_std_rel', 'high_corner_speed_std_rel', 'slow_corner_speed_pct', 'medium_corner_speed_pct', 'high_corner_speed_pct', 'slow_corner_speed_std_pct', 'medium_corner_speed_std_pct', 'high_corner_speed_std_pct']


### PART 3A: Extract Track Characteristics from 2024 Season


In [6]:
print("\n" + "="*80)
print("PART 3A: Build Track Characteristics Database")
print("="*80)

class PerformanceScoringMethod:
    """Base class for different scoring approaches."""
    
    def score_drivers(self, testing_features: pd.DataFrame) -> pd.DataFrame:
        """
        Score drivers on each characteristic.
        
        Parameters
        ----------
        testing_features : pd.DataFrame
            Features extracted from testing
        
        Returns
        -------
        pd.DataFrame
            Columns: driver_number, slow_corner_score, medium_corner_score, etc.
        """
        raise NotImplementedError


class AbsoluteDifferenceScoring(PerformanceScoringMethod):
    """Score = (value - median) in actual units."""
    
    def score_drivers(self, testing_features: pd.DataFrame) -> pd.DataFrame:
        scores = []
        
        features = {
            'slow_corner': 'slow_corner_speed',
            'medium_corner': 'medium_corner_speed', 
            'high_corner': 'high_corner_speed',
            'straight': 'avg_speed_full_throttle',
            'throttle_usage': 'pct_full_throttle'
        }
        
        for idx, row in testing_features.iterrows():
            driver = row['driver_number']
            
            score = {'driver_number': driver}
            
            for metric_name, feature_col in features.items():
                if feature_col in testing_features.columns and pd.notna(row[feature_col]):
                    median_val = testing_features[feature_col].median()
                    score[f'{metric_name}_score'] = row[feature_col] - median_val
                else:
                    score[f'{metric_name}_score'] = np.nan
            
            scores.append(score)
        
        return pd.DataFrame(scores)


class RankingScoring(PerformanceScoringMethod):
    """Score = rank (1 = best, 20 = worst)."""
    
    def score_drivers(self, testing_features: pd.DataFrame) -> pd.DataFrame:
        scores = []
        
        features = {
            'slow_corner': 'slow_corner_speed',
            'medium_corner': 'medium_corner_speed',
            'high_corner': 'high_corner_speed',
            'straight': 'avg_speed_full_throttle',
            'throttle_usage': 'pct_full_throttle'
        }
        
        for idx, row in testing_features.iterrows():
            driver = row['driver_number']
            
            score = {'driver_number': driver}
            
            for metric_name, feature_col in features.items():
                if feature_col in testing_features.columns:
                    # Rank (ascending=False: highest value = rank 1)
                    rank = testing_features[feature_col].rank(ascending=False, method='min', na_option='keep')
                    score[f'{metric_name}_score'] = rank.loc[idx]
                else:
                    score[f'{metric_name}_score'] = np.nan
            
            scores.append(score)
        
        return pd.DataFrame(scores)


class QuantileScoring(PerformanceScoringMethod):
    """Score = quantile tier (3 = top 25%, 2 = middle 50%, 1 = bottom 25%)."""
    
    def score_drivers(self, testing_features: pd.DataFrame) -> pd.DataFrame:
        scores = []
        
        features = {
            'slow_corner': 'slow_corner_speed',
            'medium_corner': 'medium_corner_speed',
            'high_corner': 'high_corner_speed',
            'straight': 'avg_speed_full_throttle',
            'throttle_usage': 'pct_full_throttle'
        }
        
        for idx, row in testing_features.iterrows():
            driver = row['driver_number']
            
            score = {'driver_number': driver}
            
            for metric_name, feature_col in features.items():
                if feature_col in testing_features.columns and pd.notna(row[feature_col]):
                    val = row[feature_col]
                    q75 = testing_features[feature_col].quantile(0.75)
                    q25 = testing_features[feature_col].quantile(0.25)
                    
                    if val >= q75:
                        tier = 3  # Top 25%
                    elif val >= q25:
                        tier = 2  # Middle 50%
                    else:
                        tier = 1  # Bottom 25%
                    
                    score[f'{metric_name}_score'] = tier
                else:
                    score[f'{metric_name}_score'] = np.nan
            
            scores.append(score)
        
        return pd.DataFrame(scores)


class ZScoreScoring(PerformanceScoringMethod):
    """Score = standardized z-score."""
    
    def score_drivers(self, testing_features: pd.DataFrame) -> pd.DataFrame:
        scores = []
        
        features = {
            'slow_corner': 'slow_corner_speed',
            'medium_corner': 'medium_corner_speed',
            'high_corner': 'high_corner_speed',
            'straight': 'avg_speed_full_throttle',
            'throttle_usage': 'pct_full_throttle'
        }
        
        for idx, row in testing_features.iterrows():
            driver = row['driver_number']
            
            score = {'driver_number': driver}
            
            for metric_name, feature_col in features.items():
                if feature_col in testing_features.columns and pd.notna(row[feature_col]):
                    mean_val = testing_features[feature_col].mean()
                    std_val = testing_features[feature_col].std()
                    
                    if std_val > 0:
                        z_score = (row[feature_col] - mean_val) / std_val
                        score[f'{metric_name}_score'] = z_score
                    else:
                        score[f'{metric_name}_score'] = 0
                else:
                    score[f'{metric_name}_score'] = np.nan
            
            scores.append(score)
        
        return pd.DataFrame(scores)


# TEST: Compare Scoring Methods

print("\nTesting scoring methods on 2024 testing data...")

# Filter to Practice 3 (most representative day)
day3_features = testing_2024[testing_2024['session_type'] == 'Practice 3']

print(f"Practice 3 drivers: {len(day3_features)}")

# Check if I have any Practice 3 data
if len(day3_features) == 0:
    print("⚠️  No Practice 3 data, using Practice 1 instead")
    day3_features = testing_2024[testing_2024['session_type'] == 'Practice 1']
    print(f"Practice 1 drivers: {len(day3_features)}")

# Initialize scoring methods
methods = {
    'Absolute Difference': AbsoluteDifferenceScoring(),
    'Ranking': RankingScoring(),
    'Quantile': QuantileScoring(),
    'Z-Score': ZScoreScoring()
}

scores_by_method = {}

print("\n" + "="*80)
print("SCORING RESULTS")
print("="*80)

for method_name, method in methods.items():
    scores = method.score_drivers(day3_features)
    scores_by_method[method_name] = scores
    
    print(f"\n{method_name}:")
    
    # Show VER's scores
    ver_row = scores[scores['driver_number'] == '1']
    if len(ver_row) > 0:
        print(f"  VER medium corners: {ver_row['medium_corner_score'].values[0]:.2f}")
        print(f"  VER straights: {ver_row['straight_score'].values[0]:.2f}")
    
    # Show top 3 in medium corners
    scores_clean = scores.dropna(subset=['medium_corner_score'])
    
    if method_name == 'Ranking':
        top3 = scores_clean.nsmallest(3, 'medium_corner_score')
        print(f"  Top 3 medium corners (by rank):")
    else:
        top3 = scores_clean.nlargest(3, 'medium_corner_score')
        print(f"  Top 3 medium corners:")
    
    for idx, row in top3.iterrows():
        print(f"    Driver {row['driver_number']:2} - Score: {row['medium_corner_score']:.2f}")

print("\n" + "="*80)
print("🟢 All scoring methods tested on 2024 testing data")


PART 3A: Build Track Characteristics Database

Testing scoring methods on 2024 testing data...
Practice 3 drivers: 0
⚠️  No Practice 3 data, using Practice 1 instead
Practice 1 drivers: 20

SCORING RESULTS

Absolute Difference:
  VER medium corners: 0.77
  VER straights: -4.40
  Top 3 medium corners:
    Driver 77 - Score: 2.60
    Driver 22 - Score: 2.11
    Driver 30 - Score: 1.95

Ranking:
  VER medium corners: 8.00
  VER straights: 18.00
  Top 3 medium corners (by rank):
    Driver 77 - Score: 1.00
    Driver 22 - Score: 2.00
    Driver 30 - Score: 3.00

Quantile:
  VER medium corners: 2.00
  VER straights: 1.00
  Top 3 medium corners:
    Driver 22 - Score: 3.00
    Driver 24 - Score: 3.00
    Driver 30 - Score: 3.00

Z-Score:
  VER medium corners: 0.35
  VER straights: -0.77
  Top 3 medium corners:
    Driver 77 - Score: 2.10
    Driver 22 - Score: 1.63
    Driver 30 - Score: 1.48

🟢 All scoring methods tested on 2024 testing data


### PART 3B: Scoring Method Comparison Framework (COMPLETE REPLACEMENT)

In [7]:
print("\n" + "="*80)
print("PART 3B: Scoring Method Comparison Framework")
print("="*80)

class PerformanceScoringMethod:
    """Base class for different scoring approaches."""
    
    def score_drivers(self, testing_features: pd.DataFrame) -> pd.DataFrame:
        """Score drivers on each characteristic."""
        raise NotImplementedError


class AbsoluteDifferenceScoring(PerformanceScoringMethod):
    """Score = (value - median) in actual units."""
    
    def score_drivers(self, testing_features: pd.DataFrame) -> pd.DataFrame:
        scores = []
        
        features = {
            'slow_corner': 'slow_corner_speed',
            'medium_corner': 'medium_corner_speed', 
            'high_corner': 'high_corner_speed',
            'straight': 'avg_speed_full_throttle',
            'throttle_usage': 'pct_full_throttle'
        }
        
        for idx, row in testing_features.iterrows():
            driver = row['driver_number']
            score = {'driver_number': driver}
            
            for metric_name, feature_col in features.items():
                if feature_col in testing_features.columns and pd.notna(row[feature_col]):
                    median_val = testing_features[feature_col].median()
                    score[f'{metric_name}_score'] = row[feature_col] - median_val
                else:
                    score[f'{metric_name}_score'] = np.nan
            
            scores.append(score)
        
        return pd.DataFrame(scores)


class RankingScoring(PerformanceScoringMethod):
    """Score = rank (1 = best, 20 = worst)."""
    
    def score_drivers(self, testing_features: pd.DataFrame) -> pd.DataFrame:
        scores = []
        
        features = {
            'slow_corner': 'slow_corner_speed',
            'medium_corner': 'medium_corner_speed',
            'high_corner': 'high_corner_speed',
            'straight': 'avg_speed_full_throttle',
            'throttle_usage': 'pct_full_throttle'
        }
        
        for idx, row in testing_features.iterrows():
            driver = row['driver_number']
            score = {'driver_number': driver}
            
            for metric_name, feature_col in features.items():
                if feature_col in testing_features.columns:
                    rank = testing_features[feature_col].rank(ascending=False, method='min', na_option='keep')
                    score[f'{metric_name}_score'] = rank.loc[idx]
                else:
                    score[f'{metric_name}_score'] = np.nan
            
            scores.append(score)
        
        return pd.DataFrame(scores)


class QuantileScoring(PerformanceScoringMethod):
    """Score = quantile tier (3 = top 25%, 2 = middle 50%, 1 = bottom 25%)."""
    
    def score_drivers(self, testing_features: pd.DataFrame) -> pd.DataFrame:
        scores = []
        
        features = {
            'slow_corner': 'slow_corner_speed',
            'medium_corner': 'medium_corner_speed',
            'high_corner': 'high_corner_speed',
            'straight': 'avg_speed_full_throttle',
            'throttle_usage': 'pct_full_throttle'
        }
        
        for idx, row in testing_features.iterrows():
            driver = row['driver_number']
            score = {'driver_number': driver}
            
            for metric_name, feature_col in features.items():
                if feature_col in testing_features.columns and pd.notna(row[feature_col]):
                    val = row[feature_col]
                    q75 = testing_features[feature_col].quantile(0.75)
                    q25 = testing_features[feature_col].quantile(0.25)
                    
                    if val >= q75:
                        tier = 3
                    elif val >= q25:
                        tier = 2
                    else:
                        tier = 1
                    
                    score[f'{metric_name}_score'] = tier
                else:
                    score[f'{metric_name}_score'] = np.nan
            
            scores.append(score)
        
        return pd.DataFrame(scores)


class ZScoreScoring(PerformanceScoringMethod):
    """Score = standardized z-score."""
    
    def score_drivers(self, testing_features: pd.DataFrame) -> pd.DataFrame:
        scores = []
        
        features = {
            'slow_corner': 'slow_corner_speed',
            'medium_corner': 'medium_corner_speed',
            'high_corner': 'high_corner_speed',
            'straight': 'avg_speed_full_throttle',
            'throttle_usage': 'pct_full_throttle'
        }
        
        for idx, row in testing_features.iterrows():
            driver = row['driver_number']
            score = {'driver_number': driver}
            
            for metric_name, feature_col in features.items():
                if feature_col in testing_features.columns and pd.notna(row[feature_col]):
                    mean_val = testing_features[feature_col].mean()
                    std_val = testing_features[feature_col].std()
                    
                    if std_val > 0:
                        z_score = (row[feature_col] - mean_val) / std_val
                        score[f'{metric_name}_score'] = z_score
                    else:
                        score[f'{metric_name}_score'] = 0
                else:
                    score[f'{metric_name}_score'] = np.nan
            
            scores.append(score)
        
        return pd.DataFrame(scores)


# TEST: Compare Scoring Methods

print("\nLoading 2024 testing features...")
testing_2024 = pd.read_parquet('../data/processed/testing_files/2024_testing_features.parquet')
print(f"🟢 Loaded {len(testing_2024)} driver-sessions")

# Use Practice 3
day3_features = testing_2024[testing_2024['session_type'] == 'Practice 3']
if len(day3_features) == 0:
    day3_features = testing_2024[testing_2024['session_type'] == 'Practice 1']

print(f"\nScoring {len(day3_features)} drivers...")

methods = {
    'Absolute Difference': AbsoluteDifferenceScoring(),
    'Ranking': RankingScoring(),
    'Quantile': QuantileScoring(),
    'Z-Score': ZScoreScoring()
}

scores_by_method = {}

for method_name, method in methods.items():
    scores = method.score_drivers(day3_features)  # FIXED: score_drivers not score_teams
    scores_by_method[method_name] = scores
    
    print(f"\n{method_name}:")
    
    ver_row = scores[scores['driver_number'] == '1']
    if len(ver_row) > 0:
        print(f"  VER medium corners: {ver_row['medium_corner_score'].values[0]:.2f}")
    
    scores_clean = scores.dropna(subset=['medium_corner_score'])
    
    if method_name == 'Ranking':
        top3 = scores_clean.nsmallest(3, 'medium_corner_score')
    else:
        top3 = scores_clean.nlargest(3, 'medium_corner_score')
    
    print(f"  Top 3 medium corners:")
    for idx, row in top3.iterrows():
        print(f"    Driver {row['driver_number']:2} - {row['medium_corner_score']:.2f}")

print("\n🟢 All scoring methods tested")


PART 3B: Scoring Method Comparison Framework

Loading 2024 testing features...
🟢 Loaded 60 driver-sessions

Scoring 20 drivers...

Absolute Difference:
  VER medium corners: 0.77
  Top 3 medium corners:
    Driver 77 - 2.60
    Driver 22 - 2.11
    Driver 30 - 1.95

Ranking:
  VER medium corners: 8.00
  Top 3 medium corners:
    Driver 77 - 1.00
    Driver 22 - 2.00
    Driver 30 - 3.00

Quantile:
  VER medium corners: 2.00
  Top 3 medium corners:
    Driver 22 - 3.00
    Driver 24 - 3.00
    Driver 30 - 3.00

Z-Score:
  VER medium corners: 0.35
  Top 3 medium corners:
    Driver 77 - 2.10
    Driver 22 - 1.63
    Driver 30 - 1.48

🟢 All scoring methods tested


#### INVESTIGATE: Why is LEC +40 km/h?

In [8]:
print("\n" + "="*80)
print("INVESTIGATION: LEC Medium Corner Anomaly")
print("="*80)

# Check raw values
print("\nRaw medium corner speeds (Day 3):")
medium_speeds = day3_features[['driver_number', 'medium_corner_speed']].sort_values(
    'medium_corner_speed', ascending=False
)
print(medium_speeds.to_string(index=False))

print(f"\nMedian: {day3_features['medium_corner_speed'].median():.2f} km/h")
print(f"Mean: {day3_features['medium_corner_speed'].mean():.2f} km/h")
print(f"Std: {day3_features['medium_corner_speed'].std():.2f} km/h")

# Check if LEC value is real or data issue
lec_row = day3_features[day3_features['driver_number'] == '16']
if len(lec_row) > 0:
    print(f"\nLEC's values:")
    print(f"  Medium corner: {lec_row['medium_corner_speed'].values[0]:.2f} km/h")
    print(f"  Slow corner: {lec_row['slow_corner_speed'].values[0]:.2f} km/h")
    print(f"  High corner: {lec_row['high_corner_speed'].values[0]:.2f} km/h")
    print(f"  Straight: {lec_row['avg_speed_full_throttle'].values[0]:.2f} km/h")
    print(f"  Clean laps: {lec_row['clean_laps'].values[0]}")

# Check for missing data
missing_count = day3_features['medium_corner_speed'].isna().sum()
print(f"\nMissing data: {missing_count}/{len(day3_features)} drivers")

print("\n" + "="*80)
print("Hypothesis: Outlier or missing data issue?")
print("="*80)


INVESTIGATION: LEC Medium Corner Anomaly

Raw medium corner speeds (Day 3):
driver_number  medium_corner_speed
           77           151.365280
           22           150.871458
           30           150.715894
           24           150.219497
           44           149.936309
           20           149.866616
           27           149.610068
            1           149.534772
           11           149.268857
           43           148.897048
            4           148.631020
           14           148.610712
           55           148.594428
           18           148.541777
           10           148.445143
           31           148.237620
           63           148.157541
           23           148.122323
           81           147.953032
           16           147.854226

Median: 148.76 km/h
Mean: 149.17 km/h
Std: 1.04 km/h

LEC's values:
  Medium corner: 147.85 km/h
  Slow corner: 89.70 km/h
  High corner: 225.40 km/h
  Straight: 251.87 km/h
  Clean laps:

### PART 3C: Outlier Handling Comparison

In [9]:
print("\n" + "="*80)
print("PART 3C: Test Outlier Handling")
print("="*80)

from scipy.stats import median_abs_deviation

def remove_outliers_mad(df: pd.DataFrame, feature: str, n_mad: float = 3.0) -> pd.DataFrame:
    """
    Remove outliers using Median Absolute Deviation.
    
    Parameters
    ----------
    df : pd.DataFrame
        Data
    feature : str
        Column to check for outliers
    n_mad : float
        Number of MADs from median to consider outlier
    
    Returns
    -------
    pd.DataFrame
        Data with outliers removed
    """
    values = df[feature].dropna()
    median = values.median()
    mad = median_abs_deviation(values, nan_policy='omit')
    
    # Define outlier bounds
    lower_bound = median - n_mad * mad
    upper_bound = median + n_mad * mad
    
    # Filter
    mask = (df[feature] >= lower_bound) & (df[feature] <= upper_bound)
    
    removed = len(df) - mask.sum()
    if removed > 0:
        removed_drivers = df[~mask]['driver_number'].tolist()
        print(f"  Removed {removed} outliers in {feature}: {removed_drivers}")
        print(f"    Bounds: [{lower_bound:.1f}, {upper_bound:.1f}]")
    
    return df[mask]

# Test on medium corners
print("\nTesting outlier removal on medium corners:")
print(f"Original: {len(day3_features)} drivers")

day3_robust = remove_outliers_mad(
    day3_features.copy(), 
    'medium_corner_speed',
    n_mad=3.0
)

print(f"After outlier removal: {len(day3_robust)} drivers")

# Score both versions
print("\n" + "="*80)
print("Comparison: Raw vs Robust")
print("="*80)

for dataset_name, dataset in [('Raw', day3_features), ('Robust', day3_robust)]:
    print(f"\n{dataset_name} Data ({len(dataset)} drivers):")
    
    # Use Absolute Difference scoring
    scorer = AbsoluteDifferenceScoring()
    scores = scorer.score_drivers(dataset)
    
    # Show top 3
    scores_clean = scores.dropna(subset=['medium_corner_score'])
    top3 = scores_clean.nlargest(3, 'medium_corner_score')
    
    print(f"  Top 3 medium corners:")
    for idx, row in top3.iterrows():
        print(f"    Driver {row['driver_number']:2} - {row['medium_corner_score']:+6.2f} km/h")


PART 3C: Test Outlier Handling

Testing outlier removal on medium corners:
Original: 20 drivers
  Removed 1 outliers in medium_corner_speed: ['77']
    Bounds: [146.6, 150.9]
After outlier removal: 19 drivers

Comparison: Raw vs Robust

Raw Data (20 drivers):
  Top 3 medium corners:
    Driver 77 -  +2.60 km/h
    Driver 22 -  +2.11 km/h
    Driver 30 -  +1.95 km/h

Robust Data (19 drivers):
  Top 3 medium corners:
    Driver 22 -  +2.24 km/h
    Driver 30 -  +2.08 km/h
    Driver 24 -  +1.59 km/h


In [10]:
def predict_race_simple(
    driver_scores: pd.DataFrame,
    track_chars: dict,
    bayesian_priors: dict
) -> pd.DataFrame:
    """
    Simple prediction: weighted sum of scores by track characteristics.
    
    track_chars should have keys like:
      'medium_corner_time_pct', 'slow_corner_time_pct', etc.
    """
    predictions = []
    
    for idx, row in driver_scores.iterrows():
        driver_num = row['driver_number']
        
        # Weighted sum of scores
        testing_signal = (
            track_chars.get('medium_corner_time_pct', 0.4) * row.get('medium_corner_score', 0) +
            track_chars.get('slow_corner_time_pct', 0.2) * row.get('slow_corner_score', 0) +
            track_chars.get('high_corner_time_pct', 0.2) * row.get('high_corner_score', 0) +
            track_chars.get('straight_time_pct', 0.2) * row.get('straight_score', 0)
        )
        
        # Get Bayesian prior
        prior_mu = bayesian_priors[driver_num].mu if driver_num in bayesian_priors else 10.0
        
        # Combine: 90% prior, 10% testing
        final_rating = 0.9 * prior_mu + 0.1 * testing_signal
        
        predictions.append({
            'driver_number': driver_num,
            'rating': final_rating
        })
    
    df = pd.DataFrame(predictions)
    df = df.sort_values('rating', ascending=False).reset_index(drop=True)
    df['predicted_position'] = range(1, len(df) + 1)
    
    return df


def predict_race_fixed(
    driver_scores: pd.DataFrame,
    track_chars: dict,
    bayesian_priors: dict
) -> pd.DataFrame:
    """
    Fixed prediction with proper z-score normalization.
    """
    # Normalize scores to z-scores
    scores_normalized = driver_scores.copy()
    
    score_cols = ['slow_corner_score', 'medium_corner_score', 
                  'high_corner_score', 'straight_score']
    
    for col in score_cols:
        if col in scores_normalized.columns:
            mean = scores_normalized[col].mean()
            std = scores_normalized[col].std()
            if std > 0:
                scores_normalized[f'{col}_z'] = (scores_normalized[col] - mean) / std
            else:
                scores_normalized[f'{col}_z'] = 0
    
    predictions = []
    
    for idx, row in scores_normalized.iterrows():
        driver_num = row['driver_number']
        
        # Track-weighted z-scores
        testing_signal = (
            track_chars.get('medium_corner_time_pct', 0.4) * row.get('medium_corner_score_z', 0) +
            track_chars.get('slow_corner_time_pct', 0.2) * row.get('slow_corner_score_z', 0) +
            track_chars.get('high_corner_time_pct', 0.2) * row.get('high_corner_score_z', 0) +
            track_chars.get('straight_time_pct', 0.2) * row.get('straight_score_z', 0)
        )
        
        # Get Bayesian prior
        prior_mu = bayesian_priors[driver_num].mu if driver_num in bayesian_priors else 10.0
        
        # Combine: 90% prior, 10% testing
        final_rating = 0.9 * (21 - prior_mu) + 0.1 * testing_signal
        
        predictions.append({
            'driver_number': driver_num,
            'rating': final_rating
        })
    
    df = pd.DataFrame(predictions)
    df = df.sort_values('rating').reset_index(drop=True)
    df['predicted_position'] = range(1, len(df) + 1)
    
    return df

print("🟢 Prediction functions defined")

def predict_prior_only(bayesian_priors: dict) -> pd.DataFrame:
    """
    Baseline: Predict using ONLY Bayesian priors, no testing data.
    
    This is the "do nothing" baseline - just rank drivers by championship standings.
    """
    predictions = []
    
    for driver_num, prior in bayesian_priors.items():
        # Lower μ = better expected position
        predictions.append({
            'driver_number': driver_num,
            'predicted_position': int(21 - prior.mu)
        })
    
    df = pd.DataFrame(predictions)
    df = df.sort_values('predicted_position').reset_index(drop=True)
    
    return df

print("🟢 All prediction functions defined")


🟢 Prediction functions defined
🟢 All prediction functions defined


### PART 3D: Simple Prediction Test - Bahrain 2024

In [11]:
print("\n" + "="*80)
print("PART 3D: Predict Bahrain 2024 Race")
print("="*80)

print("\nPredicting Bahrain 2024 Race...")

# Define simple track characteristics (placeholder)
# In real implementation, extract from track database
bahrain_chars = {
    'slow_corner_time_pct': 0.2,
    'medium_corner_time_pct': 0.4,
    'high_corner_time_pct': 0.2,
    'straight_time_pct': 0.2
}



# Use Robust + Absolute Difference scoring
scorer = AbsoluteDifferenceScoring()
day3_robust = remove_outliers_mad(day3_features.copy(), 'medium_corner_speed', n_mad=3.0)
driver_scores = scorer.score_drivers(day3_robust)

# Make predictions
predictions = predict_race_simple(
    driver_scores,
    bahrain_chars,
    priors
)

print(f"\n🟢 Predicted positions for {len(predictions)} drivers")

# Load actual Bahrain 2024 race results
print("\nLoading actual Bahrain 2024 race results...")
bahrain_2024_race = fastf1.get_session(2024, 'Bahrain', 'Race')
bahrain_2024_race.load()

actual_results = bahrain_2024_race.results[['DriverNumber', 'Position', 'Abbreviation']].copy()
actual_results = actual_results[actual_results['Position'].notna()]
actual_results['DriverNumber'] = actual_results['DriverNumber'].astype(str)

# Merge predictions with actuals
comparison = predictions.merge(
    actual_results,
    left_on='driver_number',
    right_on='DriverNumber',
    how='inner'
)

comparison['error'] = abs(comparison['predicted_position'] - comparison['Position'])

# Calculate MAE
mae = comparison['error'].mean()

print("\n" + "="*80)
print("RESULTS: Testing Profile → Race Prediction")
print("="*80)

print(f"\nMean Absolute Error: {mae:.2f} positions")
print(f"Median Error: {comparison['error'].median():.2f} positions")

# Show top 10 comparison
print(f"\nTop 10 Predicted vs Actual:")
print(comparison.nsmallest(10, 'predicted_position')[
    ['Abbreviation', 'predicted_position', 'Position', 'error']
].to_string(index=False))

# Show biggest errors
print(f"\nBiggest Prediction Errors:")
print(comparison.nlargest(5, 'error')[
    ['Abbreviation', 'predicted_position', 'Position', 'error']
].to_string(index=False))

# Baseline comparison
print(f"\n" + "="*80)
print("Baseline: Random Guess")
print("="*80)

import random

random.seed(42)
random_positions = list(range(1, len(comparison) + 1))
random.shuffle(random_positions)
comparison['random_prediction'] = random_positions
comparison['random_error'] = abs(comparison['random_prediction'] - comparison['Position'])
random_mae = comparison['random_error'].mean()

print(f"Random MAE: {random_mae:.2f} positions")
print(f"Our MAE: {mae:.2f} positions")
print(f"Improvement: {random_mae - mae:.2f} positions ({(random_mae - mae)/random_mae*100:.1f}%)")

if mae < 5:
    print("\n🟢 MAE < 5 - Concept works! Ready to optimize.")
elif mae < 7:
    print("\n⚠️  MAE 5-7 - Promising, needs refinement.")
else:
    print("\n🔴 MAE > 7 - Concept needs rethinking.")

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...



PART 3D: Predict Bahrain 2024 Race

Predicting Bahrain 2024 Race...
  Removed 1 outliers in medium_corner_speed: ['77']
    Bounds: [146.6, 150.9]

🟢 Predicted positions for 19 drivers

Loading actual Bahrain 2024 race results...


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']



RESULTS: Testing Profile → Race Prediction

Mean Absolute Error: 3.41 positions
Median Error: 2.00 positions

Top 10 Predicted vs Actual:
Abbreviation  predicted_position  Position  error
         NOR                   1       6.0    5.0
         VER                   2       1.0    1.0
         PIA                   3       8.0    5.0
         RUS                   4       5.0    1.0
         LEC                   5       4.0    1.0
         HAM                   6       7.0    1.0
         SAI                   7       3.0    4.0
         ALB                   8      15.0    7.0
         ALO                   9       9.0    0.0
         MAG                  10      12.0    2.0

Biggest Prediction Errors:
Abbreviation  predicted_position  Position  error
         PER                  14       2.0   12.0
         STR                  18      10.0    8.0
         ALB                   8      15.0    7.0
         NOR                   1       6.0    5.0
         PIA                   3 

In [12]:
# FIX: Proper Prediction Function

print("\n" + "="*80)
print("FIXING: Proper Normalization")
print("="*80)

def predict_race_fixed(
    driver_scores: pd.DataFrame,
    track_chars: dict,
    bayesian_priors: dict
) -> pd.DataFrame:
    """
    Fixed prediction with proper normalization.
    
    Steps:
    1. Convert scores to z-scores (dimensionless)
    2. Weight by track characteristics
    3. Combine with Bayesian priors (90% prior, 10% testing)
    """
    # Normalize all scores to z-scores
    features = ['slow_corner_score', 'medium_corner_score', 
                'high_corner_score', 'straight_score']
    
    scores_normalized = driver_scores.copy()
    
    for feature in features:
        mean = scores_normalized[feature].mean()
        std = scores_normalized[feature].std()
        
        if std > 0:
            scores_normalized[f'{feature}_z'] = (scores_normalized[feature] - mean) / std
        else:
            scores_normalized[f'{feature}_z'] = 0
    
    predictions = []
    
    for idx, row in scores_normalized.iterrows():
        driver_num = row['driver_number']
        
        # Track-weighted performance (now dimensionless z-scores)
        testing_signal = (
            track_chars['medium_corner_time_pct'] * row['medium_corner_score_z'] +
            track_chars['slow_corner_time_pct'] * row['slow_corner_score_z'] +
            track_chars['high_corner_time_pct'] * row['high_corner_score_z'] +
            track_chars['straight_time_pct'] * row['straight_score_z']
        )
        
        # Get Bayesian prior (this is the MAIN signal)
        if driver_num in bayesian_priors:
            prior_mu = bayesian_priors[driver_num].mu
        else:
            prior_mu = 10
        
        # Combine: 90% prior, 10% testing (testing is unreliable!)
        combined_rating = 0.9 * prior_mu + 0.1 * testing_signal
        
        predictions.append({
            'driver_number': driver_num,
            'testing_signal': testing_signal,
            'prior_mu': prior_mu,
            'combined_rating': combined_rating
        })
    
    pred_df = pd.DataFrame(predictions)
    pred_df = pred_df.sort_values('combined_rating', ascending=False)
    pred_df['predicted_position'] = range(1, len(pred_df) + 1)
    
    return pred_df


# RE-TEST: With Fixed Prediction

print("\nRe-predicting with FIXED normalization...")

predictions_fixed = predict_race_fixed(
    driver_scores,
    bahrain_chars,
    priors
)

# Merge with actuals
comparison_fixed = predictions_fixed.merge(
    actual_results,
    left_on='driver_number',
    right_on='DriverNumber',
    how='inner'
)

comparison_fixed['error'] = abs(comparison_fixed['predicted_position'] - comparison_fixed['Position'])
mae_fixed = comparison_fixed['error'].mean()

print("\n" + "="*80)
print("FIXED RESULTS")
print("="*80)

print(f"\nMAE: {mae_fixed:.2f} positions")

print(f"\nTop 10 Predicted vs Actual:")
print(comparison_fixed.nsmallest(10, 'predicted_position')[
    ['driver_number', 'predicted_position', 'Position', 'error']
].to_string(index=False))

print(f"\nWho did I predict P1?")
winner_pred = comparison_fixed[comparison_fixed['predicted_position'] == 1]
print(f"  Driver {winner_pred['driver_number'].values[0]} (actual: P{winner_pred['Position'].values[0]:.0f})")

print(f"\nWho actually won?")
actual_winner = comparison_fixed[comparison_fixed['Position'] == 1]
print(f"  Driver {actual_winner['driver_number'].values[0]} (I predicted: P{actual_winner['predicted_position'].values[0]})")


FIXING: Proper Normalization

Re-predicting with FIXED normalization...

FIXED RESULTS

MAE: 3.35 positions

Top 10 Predicted vs Actual:
driver_number  predicted_position  Position  error
            1                   1       1.0    0.0
            4                   2       6.0    4.0
           81                   3       8.0    5.0
           63                   4       5.0    1.0
           16                   5       4.0    1.0
           44                   6       7.0    1.0
           55                   7       3.0    4.0
           14                   8       9.0    1.0
           23                   9      15.0    6.0
           20                  10      12.0    2.0

Who did I predict P1?
  Driver 1 (actual: P1)

Who actually won?
  Driver 1 (I predicted: P1)


I used FUTURE data (2025) to predict PAST (2024).
Why it works well:

NOR won 2025 championship → I predicted him P1 in 2024
VER 2nd in 2025 → I predicted him P2 in 2024
But in reality: VER won 2024 championship, NOR was P2!


In [13]:
# CRITICAL FIX: Use 2023 Standings as Priors for 2024

print("\n" + "="*80)
print("CRITICAL FIX: Temporal Validation")
print("="*80)

def initialize_2023_standings_priors() -> Dict[str, DriverPrior]:
    """
    2023 Final Championship Standings (correct priors for 2024).
    
    2023 Results:
    1. VER - 575 pts (dominant)
    2. PER - 285 pts
    3. HAM - 234 pts
    4. ALO - 206 pts
    5. SAI - 200 pts
    6. RUS - 175 pts
    7. LEC - 206 pts
    8. NOR - 205 pts
    9. PIA - 97 pts (rookie)
    ...
    """
    priors_2023 = {
        # Top tier 2023
        '1': DriverPrior('1', 'VER', 'Red Bull Racing', 'top', 'elite', mu=20, sigma=3),  # Dominant
        '11': DriverPrior('11', 'PER', 'Red Bull Racing', 'top', 'experienced', mu=16, sigma=4),
        
        # Upper midfield 2023
        '44': DriverPrior('44', 'HAM', 'Mercedes', 'midfield', 'elite', mu=15, sigma=4),
        '14': DriverPrior('14', 'ALO', 'Aston Martin', 'midfield', 'elite', mu=15, sigma=4),
        '55': DriverPrior('55', 'SAI', 'Ferrari', 'midfield', 'experienced', mu=15, sigma=4),
        '63': DriverPrior('63', 'RUS', 'Mercedes', 'midfield', 'experienced', mu=14, sigma=4),
        '16': DriverPrior('16', 'LEC', 'Ferrari', 'midfield', 'elite', mu=15, sigma=4),
        '4': DriverPrior('4', 'NOR', 'McLaren', 'midfield', 'experienced', mu=15, sigma=4),
        
        # Promising 2023
        '81': DriverPrior('81', 'PIA', 'McLaren', 'midfield', 'rookie', mu=12, sigma=5),  # Strong rookie
        
        # Lower midfield 2023
        '10': DriverPrior('10', 'GAS', 'Alpine', 'backmarker', 'experienced', mu=10, sigma=5),
        '23': DriverPrior('23', 'ALB', 'Williams', 'backmarker', 'experienced', mu=10, sigma=5),
        '18': DriverPrior('18', 'STR', 'Aston Martin', 'backmarker', 'experienced', mu=9, sigma=5),
        '31': DriverPrior('31', 'OCO', 'Alpine', 'backmarker', 'experienced', mu=9, sigma=5),
        '20': DriverPrior('20', 'MAG', 'Haas', 'backmarker', 'experienced', mu=8, sigma=6),
        '27': DriverPrior('27', 'HUL', 'Haas', 'backmarker', 'experienced', mu=8, sigma=6),
        '22': DriverPrior('22', 'TSU', 'AlphaTauri', 'backmarker', 'experienced', mu=8, sigma=6),
        '24': DriverPrior('24', 'ZHO', 'Alfa Romeo', 'backmarker', 'experienced', mu=7, sigma=6),
        '77': DriverPrior('77', 'BOT', 'Alfa Romeo', 'backmarker', 'experienced', mu=8, sigma=6),
        '3': DriverPrior('3', 'RIC', 'AlphaTauri', 'backmarker', 'experienced', mu=9, sigma=5),
        '2': DriverPrior('2', 'SAR', 'Williams', 'backmarker', 'rookie', mu=6, sigma=7),
    }
    
    return priors_2023

# Initialize correct priors
priors_2023 = initialize_2023_standings_priors()

print("\n2023 Championship-based priors:")
print("Top tier:")
for num in ['1', '11']:
    p = priors_2023[num]
    print(f"  {p.driver_code}: μ={p.mu}, σ={p.sigma}")

print("\nMcLaren in 2023 (before breakthrough):")
for num in ['4', '81']:
    if num in priors_2023:
        p = priors_2023[num]
        print(f"  {p.driver_code}: μ={p.mu}, σ={p.sigma}")

# Re-predict with correct temporal priors
predictions_temporal = predict_race_fixed(
    driver_scores,
    bahrain_chars,
    priors_2023
)

comparison_temporal = predictions_temporal.merge(
    actual_results,
    left_on='driver_number',
    right_on='DriverNumber',
    how='inner'
)

comparison_temporal['error'] = abs(comparison_temporal['predicted_position'] - comparison_temporal['Position'])
mae_temporal = comparison_temporal['error'].mean()

print("\n" + "="*80)
print("PROPER TEMPORAL VALIDATION")
print("="*80)

print(f"\nMAE with 2023 priors: {mae_temporal:.2f} positions")
print(f"MAE with 2025 priors (cheating): {mae_fixed:.2f} positions")
print(f"Difference: {mae_temporal - mae_fixed:+.2f} positions")

print(f"\nTop 10 Predicted vs Actual:")
print(comparison_temporal.nsmallest(10, 'predicted_position')[
    ['driver_number', 'predicted_position', 'Position', 'error']
].to_string(index=False))

print(f"\nWho did I predict P1?")
winner_pred = comparison_temporal[comparison_temporal['predicted_position'] == 1]
if len(winner_pred) > 0:
    print(f"  Driver {winner_pred['driver_number'].values[0]} (actual: P{winner_pred['Position'].values[0]:.0f})")

print(f"\nActual winner:")
actual_winner = comparison_temporal[comparison_temporal['Position'] == 1]
if len(actual_winner) > 0:
    print(f"  Driver {actual_winner['driver_number'].values[0]} (I predicted: P{actual_winner['predicted_position'].values[0]})")

if mae_temporal < 5:
    print(f"\n🟢 MAE < 5 with proper validation - System works!")
else:
    print(f"\n⚠️  MAE {mae_temporal:.1f} with proper validation - Needs improvement")


CRITICAL FIX: Temporal Validation

2023 Championship-based priors:
Top tier:
  VER: μ=20, σ=3
  PER: μ=16, σ=4

McLaren in 2023 (before breakthrough):
  NOR: μ=15, σ=4
  PIA: μ=12, σ=5

PROPER TEMPORAL VALIDATION

MAE with 2023 priors: 2.88 positions
MAE with 2025 priors (cheating): 3.35 positions
Difference: -0.47 positions

Top 10 Predicted vs Actual:
driver_number  predicted_position  Position  error
            1                   1       1.0    0.0
           11                   2       2.0    0.0
           44                   3       7.0    4.0
            4                   4       6.0    2.0
           55                   5       3.0    2.0
           16                   6       4.0    2.0
           14                   7       9.0    2.0
           63                   8       5.0    3.0
           81                   9       8.0    1.0
           23                  12      15.0    3.0

Who did I predict P1?
  Driver 1 (actual: P1)

Actual winner:
  Driver 1 (I predi

The Hidden Problem
MAE 2.61 is suspiciously perfect:

VER P1 → Actual P1 ✅ (Error: 0)<br>
PER P2 → Actual P2 ✅ (Error: 0)<br>
NOR P6 → Actual P6 ✅ (Error: 0)<br>
PIA P8 → Actual P8 ✅ (Error: 0)<br>

But here's the issue:
We're weighting 90% prior, 10% testing. That means we're basically just predicting based on 2023 championship standings, and testing data barely matters!


### CRITICAL TEST: Does Testing Data Add Value?


In [14]:
print("\n" + "="*80)
print("ABLATION TEST: Prior Only vs Prior + Testing")
print("="*80)

def predict_prior_only(priors: dict) -> pd.DataFrame:
    """Predict using ONLY priors (ignore testing completely)."""
    predictions = []
    
    for driver_num, prior in priors.items():
        predictions.append({
            'driver_number': driver_num,
            'combined_rating': prior.mu
        })
    
    pred_df = pd.DataFrame(predictions)
    pred_df = pred_df.sort_values('combined_rating', ascending=False)
    pred_df['predicted_position'] = range(1, len(pred_df) + 1)
    
    return pred_df

# Test 1: Prior only (0% testing)
print("\nTest 1: Prior Only (2023 standings)")
pred_prior_only = predict_prior_only(priors_2023)
comp_prior = pred_prior_only.merge(actual_results, left_on='driver_number', right_on='DriverNumber', how='inner')
comp_prior['error'] = abs(comp_prior['predicted_position'] - comp_prior['Position'])
mae_prior = comp_prior['error'].mean()

print(f"MAE (Prior Only): {mae_prior:.2f}")

# Test 2: Current system (90% prior, 10% testing)
print(f"MAE (90% Prior + 10% Testing): {mae_temporal:.2f}")

# Test 3: Balanced (50% prior, 50% testing)
def predict_balanced(
    driver_scores: pd.DataFrame,
    track_chars: dict,
    bayesian_priors: dict,
    prior_weight: float = 0.5
) -> pd.DataFrame:
    """Predict with adjustable weights."""
    scores_normalized = driver_scores.copy()
    
    features = ['slow_corner_score', 'medium_corner_score', 'high_corner_score', 'straight_score']
    
    for feature in features:
        mean = scores_normalized[feature].mean()
        std = scores_normalized[feature].std()
        if std > 0:
            scores_normalized[f'{feature}_z'] = (scores_normalized[feature] - mean) / std
        else:
            scores_normalized[f'{feature}_z'] = 0
    
    predictions = []
    
    for idx, row in scores_normalized.iterrows():
        driver_num = row['driver_number']
        
        testing_signal = (
            track_chars['medium_corner_time_pct'] * row['medium_corner_score_z'] +
            track_chars['slow_corner_time_pct'] * row['slow_corner_score_z'] +
            track_chars['high_corner_time_pct'] * row['high_corner_score_z'] +
            track_chars['straight_time_pct'] * row['straight_score_z']
        )
        
        if driver_num in bayesian_priors:
            prior_mu = bayesian_priors[driver_num].mu
        else:
            prior_mu = 10
        
        # Adjustable weighting
        combined_rating = prior_weight * prior_mu + (1 - prior_weight) * testing_signal
        
        predictions.append({
            'driver_number': driver_num,
            'combined_rating': combined_rating
        })
    
    pred_df = pd.DataFrame(predictions)
    pred_df = pred_df.sort_values('combined_rating', ascending=False)
    pred_df['predicted_position'] = range(1, len(pred_df) + 1)
    
    return pred_df

print("\nTest 3: Balanced (50% Prior + 50% Testing)")
pred_balanced = predict_balanced(driver_scores, bahrain_chars, priors_2023, prior_weight=0.5)
comp_balanced = pred_balanced.merge(actual_results, left_on='driver_number', right_on='DriverNumber', how='inner')
comp_balanced['error'] = abs(comp_balanced['predicted_position'] - comp_balanced['Position'])
mae_balanced = comp_balanced['error'].mean()

print(f"MAE (50/50): {mae_balanced:.2f}")

# Test 4: Testing-heavy (10% prior, 90% testing)
print("\nTest 4: Testing-Heavy (10% Prior + 90% Testing)")
pred_testing = predict_balanced(driver_scores, bahrain_chars, priors_2023, prior_weight=0.1)
comp_testing = pred_testing.merge(actual_results, left_on='driver_number', right_on='DriverNumber', how='inner')
comp_testing['error'] = abs(comp_testing['predicted_position'] - comp_testing['Position'])
mae_testing = comp_testing['error'].mean()

print(f"MAE (10% Prior + 90% Testing): {mae_testing:.2f}")

# Summary
print("\n" + "="*80)
print("SUMMARY: Does Testing Data Help?")
print("="*80)

results = [
    ('Prior Only (0% testing)', mae_prior),
    ('Conservative (90% prior)', mae_temporal),
    ('Balanced (50% prior)', mae_balanced),
    ('Testing-Heavy (10% prior)', mae_testing),
]

print("\nMAE by weighting strategy:")
for name, mae in sorted(results, key=lambda x: x[1]):
    improvement = mae_prior - mae
    print(f"  {name:30} MAE: {mae:.2f}  (Δ{improvement:+.2f})")

best_method = min(results, key=lambda x: x[1])
print(f"\n{'🟢' if best_method[1] < mae_prior else '🔴'} Best: {best_method[0]} (MAE: {best_method[1]:.2f})")

if best_method[0] == 'Prior Only (0% testing)':
    print("\n⚠️  WARNING: Testing data doesn't improve predictions!")
    print("   Possible reasons:")
    print("   - Testing too unreliable (sandbagging)")
    print("   - Track characteristics not discriminative enough")
    print("   - Need more sophisticated modeling")
else:
    print(f"\n🟢 Testing data improves MAE by {mae_prior - best_method[1]:.2f} positions")


ABLATION TEST: Prior Only vs Prior + Testing

Test 1: Prior Only (2023 standings)
MAE (Prior Only): 2.60
MAE (90% Prior + 10% Testing): 2.88

Test 3: Balanced (50% Prior + 50% Testing)
MAE (50/50): 2.65

Test 4: Testing-Heavy (10% Prior + 90% Testing)
MAE (10% Prior + 90% Testing): 3.06

SUMMARY: Does Testing Data Help?

MAE by weighting strategy:
  Prior Only (0% testing)        MAE: 2.60  (Δ+0.00)
  Balanced (50% prior)           MAE: 2.65  (Δ-0.05)
  Conservative (90% prior)       MAE: 2.88  (Δ-0.28)
  Testing-Heavy (10% prior)      MAE: 3.06  (Δ-0.46)

🔴 Best: Prior Only (0% testing) (MAE: 2.60)

⚠️  WARNING: Testing data doesn't improve predictions!
   Possible reasons:
   - Testing too unreliable (sandbagging)
   - Track characteristics not discriminative enough
   - Need more sophisticated modeling


## Key Insight: Testing Data in Stable vs Reset Regulations

### What I Learned

Testing data **hurts** predictions in stable regulations (2024):
- Prior only: 2.60 MAE
- With testing: 2.88 MAE (11% worse)

**Why?** In stable regulations, championship standings are extremely predictive. 
Teams know their pecking order. Testing adds noise because:

1. **Sandbagging** - Teams hide true pace (especially top teams)
2. **Program variance** - Long runs vs quali sims vs reliability tests
3. **Strategic opacity** - Can't tell fuel loads, engine modes, tire compounds

### What Testing CAN Tell Us

Testing is poor for **position prediction** but valuable for **car characteristics**:

Good signals:
- Slow corner performance (mechanical grip)
- Fast corner performance (aerodynamic efficiency)
- Straight-line speed (power unit, drag)
- Relative strengths (cornering god vs straight-line monster)

Bad signals:
- Absolute lap times (sandbagging)
- Direct position prediction (hidden pace)
- Race pace simulation (too many unknowns)

### The 2026 Difference

After regulation changes, this flips:

| Regulation State | Priors Quality | Testing Value |
|-----------------|----------------|---------------|
| **Stable (2024)** | Strong (2023 standings) | Low (sandbagging noise) |
| **Reset (2026)** | Weak (no valid history) | High (only signal) |

In 2026, everyone starts from zero. No one knows the pecking order. 
Testing becomes the PRIMARY signal because:
- Priors are weak (based on organizational capability only)
- Teams can't hide fundamental car characteristics
- Relative performance visible even with sandbagging

### Better Approach for 2026

Current approach: `Testing → Positions` ❌

Better approach: `Testing → Car Profile → Track Match → Adjusted Positions` 

Step 1: Extract car profiles
```
Red Bull: {slow_corners: -0.2, fast_corners: +0.8, straights: +0.5}
Ferrari:  {slow_corners: +0.3, fast_corners: +0.2, straights: +0.9}
McLaren:  {slow_corners: +0.5, fast_corners: +0.4, straights: +0.1}
```

Step 2: Match to track characteristics
```
Monaco (70% slow corners) → McLaren advantage
Monza (60% straights)     → Ferrari advantage
Silverstone (balanced)    → Red Bull advantage
```

Step 3: Adjust priors by track fit
```
LEC at Monaco: Prior μ=14 → Adjusted μ=13 (Ferrari weak in slow corners)
LEC at Monza:  Prior μ=14 → Adjusted μ=15 (Ferrari strong on straights)
```

### Actionable Next Steps

For 2024/2025 (stable regulations):
- Use priors only (championship standings)
- Skip testing data (adds noise)
- Focus on sequential learning from race results

For 2026 (regulation reset):
- Build car profile extraction from testing
- Implement sandbagging detection (fuel load proxies, % full throttle)
- Create track-characteristic matching system
- Weight testing heavily (70-90%) because priors are weak